### Dependencies ###

In [1]:
import os
import torch
import numpy as np
import time
import pandas as pd

from scipy import spatial
print(os.listdir('train'))

['lcp_multi_train.tsv', 'lcp_single_train.tsv']


### Train data ###

In [2]:
train_single_tsv = 'train/lcp_single_train.tsv'
df_train_single = pd.read_csv(train_single_tsv, sep='\t', header=0)

In [3]:
print("Data columns: \n")
print(df_train_single.columns)
print("Total corpus len: {}".format(len(df_train_single)))
print("Subcorpus len:\n")
print(df_train_single['corpus'].value_counts())

Data columns: 

Index(['id', 'corpus', 'sentence', 'token', 'complexity'], dtype='object')
Total corpus len: 7662
Subcorpus len:

biomed      2576
bible       2574
europarl    2512
Name: corpus, dtype: int64


### Test data 

In [4]:
test_single_tsv = 'test/lcp_single_test.tsv'
df_test_single = pd.read_csv(test_single_tsv, sep='\t', header=0)


In [6]:
print("Data columns: \n")
print(df_test_single.columns)
print("Total corpus len: {}".format(len(df_test_single)))
print("Subcorpus len:\n")
print(df_test_single['corpus'].value_counts())
print(os.getcwd())

Data columns: 

Index(['id', 'corpus', 'sentence', 'token', 'complexity'], dtype='object')
Total corpus len: 917
Subcorpus len:

europarl    345
biomed      289
bible       283
Name: corpus, dtype: int64
C:\Users\Simona Mihaylova


### Trial data 


In [7]:
trial_single_tsv = 'test/lcp_single_trial.tsv'
df_trial_single = pd.read_csv(trial_single_tsv, sep='\t', header=0)


In [8]:
print("Data columns: \n")
print(df_trial_single.columns)
print("Total corpus len: {}".format(len(df_trial_single)))
print("Subcorpus len:\n")
print(df_trial_single['subcorpus'].value_counts())
print(os.getcwd())

Data columns: 

Index(['id', 'subcorpus', 'sentence', 'token', 'complexity'], dtype='object')
Total corpus len: 421
Subcorpus len:

europarl    143
bible       143
biomed      135
Name: subcorpus, dtype: int64
C:\Users\Simona Mihaylova


### GloVe ###
Load the pretrained GloVe vectors and verify that the operation has been successful by some quick experiments with the embedding.  

In [12]:
glove_w2v_loc = 'InferSent/GloVe/glove.6B.300d.txt'
with open(glove_w2v_loc,  "r", encoding="utf8") as lines:
    glove_w2v = {}
    for line in lines:
        values = line.split()
        word = ''.join(values[:-300])
        vector = np.asarray(values[-300:], dtype='float32')
        glove_w2v[word.lower()] = vector
    print(len(glove_w2v)," words loaded!")

400000  words loaded!


In [13]:
def find_closest_embeddings(embedding):
    return sorted(glove_w2v.keys(), key=lambda word: spatial.distance.euclidean(glove_w2v[word.lower()], embedding))[0:5]

In [14]:
print("Demo of closest words:")
find_closest_embeddings(glove_w2v['baby'])

Demo of closest words:


['baby', 'babies', 'newborn', 'infant', 'birth']

In [15]:
print("Demo of word arithmetics:")
find_closest_embeddings(np.array(glove_w2v['king']) - np.array(glove_w2v['man']) + np.array(glove_w2v['woman']))

Demo of word arithmetics:


['king', 'queen', 'monarch', 'mother', 'princess']

### InferSent
- https://towardsdatascience.com/learning-sentence-embeddings-by-natural-language-inference-a50b4661a0b8
- https://research.fb.com/downloads/infersent/

Load InferSent model and execute some experiments.  
Currently it is using GloVe. We have tested GloVe and fastText and have chosen GloVe over fastText.

In [22]:
from InferSent.models import InferSent

In [23]:
model_pkl = 'InferSent/encoder/infersent1.pkl'
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': 1}
infer_sent_model = InferSent(params_model)
infer_sent_model.load_state_dict(torch.load(model_pkl))

<All keys matched successfully>

In [24]:
infer_sent_model.set_w2v_path(glove_w2v_loc)
infer_sent_model.build_vocab_k_words(K=100000)

# infer_sent_model.to(torch.device("cuda:0"))

Vocab size : 100000


In [25]:
infer_sent_model.encode(["This man is playing computer games"], tokenize=True)

array([[ 0.17863287,  0.08774212,  0.05200037, ...,  0.00108395,
        -0.05402661,  0.03372178]], dtype=float32)

In [26]:
def get_embedding_for_context(ctx):
    if not isinstance(ctx, list):
#       print("ctx is not list")
        ctx = [ctx]
    return infer_sent_model.encode(ctx, tokenize=True)

start = time.time()
get_embedding_for_context("This is a test sentence")
print("Time for single prediction: {}".format(time.time() - start))

get_embedding_for_context(["This is a test sentence"] * 3000)
print("Time for pred of 3000 cases: {}".format(time.time() - start))

Time for single prediction: 0.0921173095703125
Time for pred of 3000 cases: 18.23288059234619


In [27]:
from sklearn.metrics.pairwise import cosine_similarity

def measure_dist_between_ctx(c1, c2):
    e1 = get_embedding_for_context(c1)[0]
    e2 = get_embedding_for_context(c2)[0]
    #return spatial.distance.euclidean(e1, e2)
    return cosine_similarity([e1], [e2])

print(measure_dist_between_ctx("In India people are going to war.", "The family went to an indian restaurant."))
print(measure_dist_between_ctx("The baby is hungry.", "The child needs to eat."))
print(measure_dist_between_ctx("Programming takes ages to master.", "Ronaldo scored a goal against man united."))
print(measure_dist_between_ctx("At the university students go to lectures.", "Ronaldo scored a goal against man united."))
print(measure_dist_between_ctx("A soccer game with multiple males playing.", "Some men are playing a sport."))
print(measure_dist_between_ctx("The man is cooking chicken with potatoes.", "A man is driving down a lonely road."))
print(measure_dist_between_ctx("The man is cooking chicken with potatoes.", "In the restaurant they serve delicious food."))



[[0.7978083]]
[[0.8062569]]
[[0.632438]]
[[0.69138]]
[[0.8188078]]
[[0.77679014]]
[[0.84341216]]


### Handcrafted features

* Word length
* Syllable count
* Word frequency

In [117]:
import syllables
# According to the paper there are 3 handcrafted features
# - word length
# - word frequency 
# - syllable count
import csv
import math
import nltk

from collections import defaultdict
from nltk.stem.porter import PorterStemmer

reader = csv.reader(open('SUBTLEX.csv', 'r'))
frequency = defaultdict(float)
stemmer = PorterStemmer()

for row in reader:
    frequency[stemmer.stem(row[0].lower())] += float(row[5])

frequency = {k: math.log(v) for k, v in frequency.items()}

def get_handcrafted_features(word):
    word = str(word)
    return [len(word), syllables.estimate(word), frequency.get(stemmer.stem(word.lower())) or 0]


get_handcrafted_features("Basketball")



[10, 3, 3.0782334950657346]

### Load datasets

In [46]:
from torch.utils.data import Dataset

def preprocess_embeddings(dataset):
    # Preprocess all sentence embeddings for the data:
    sentence_embeddings = {}
    
    all_sentences = dataset['sentence'].tolist()

    start = time.time()
    all_sentence_embeddings = get_embedding_for_context(all_sentences)
    print("Encoding time for all sentences: {}".format(time.time() - start))
    return all_sentence_embeddings
    

class CompLexDataset(Dataset):
    global dataset_type
    
    def __init__(self, dataset_type):
        self.dataset_type = dataset_type
        
        if(self.dataset_type == 'train'):                   
            self.all_sentence_embeddings = preprocess_embeddings(df_train_single)
        elif(self.dataset_type == 'trial'):
            self.all_sentence_embeddings = preprocess_embeddings(df_trial_single)
        elif(self.dataset_type == 'test'):
            self.all_sentence_embeddings = preprocess_embeddings(df_test_single)
    
    def __len__(self):
        if(self.dataset_type == 'train'):                   
            return len(df_train_single)
        elif(self.dataset_type == 'trial'):
            return len(df_trial_single)
        elif(self.dataset_type == 'test'):
            return len(df_test_single)
        else: 
            raise Exception("Invalid dataset type.", self.dataset_type)

    
    def __getitem__(self,idx):
        start = time.time()
        if(self.dataset_type == 'train'):
            token = df_train_single.loc[idx, 'token']
            token = str(token)
            out = df_train_single.loc[idx, 'complexity']
        elif(self.dataset_type == 'trial'):
            token = df_trial_single.loc[idx, 'token']
            token = str(token)
            out = df_trial_single.loc[idx, 'complexity']
        elif(self.dataset_type == 'test'):
            token = df_test_single.loc[idx, 'token']
            token = str(token)
            out = df_test_single.loc[idx, 'complexity']
        else: 
            raise Exception("Invalid dataset type.", self.dataset_type)
        
        handcrafted_features = get_handcrafted_features(token)

        sentence_ctx = self.all_sentence_embeddings[idx]
        
        if token.lower() in glove_w2v:   
            w2v_for_token = glove_w2v[token.lower()]
        else:
            #print("Token {} not found".format(token.lower()))
            w2v_for_token = [0] * 300
        
        
        result = {
            'inp': torch.from_numpy(np.hstack((np.array(handcrafted_features), sentence_ctx, np.array(w2v_for_token))).ravel()).float(), 
            'out': torch.from_numpy(np.array([out])).float()
        }
        
        #print("Idx {} fetch time: {}".format(idx, time.time() - start))
        return result
    

In [47]:
train_dataset = CompLexDataset("train")
print("Input: ", train_dataset[5]['inp'], "Input Length: ", len(train_dataset[5]['inp']))
print("Output: ", train_dataset[5]['out'])

trial_dataset = CompLexDataset("trial")
print("Input: ", trial_dataset[5]['inp'], "Input Length: ", len(trial_dataset[5]['inp']))
print("Output: ", trial_dataset[5]['out'])

test_dataset = CompLexDataset("test")
print("Input: ", test_dataset[5]['inp'], "Input Length: ", len(test_dataset[5]['inp']))
print("Output: ", test_dataset[5]['out'])

C:\Users\Simona Mihaylova\InferSent\models.py:207: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  sentences = np.array(sentences)[idx_sort]


Encoding time for all sentences: 274.9542806148529
Input:  tensor([12.0000,  4.0000, -0.2614,  ...,  0.2898,  0.1594, -0.2284]) Input Length:  4399
Output:  tensor([0.3750])
Encoding time for all sentences: 16.22931694984436
Input:  tensor([ 4.0000,  1.0000,  4.8308,  ..., -0.1070,  0.1319, -0.1578]) Input Length:  4399
Output:  tensor([0.0250])
Encoding time for all sentences: 34.14873957633972
Input:  tensor([ 4.0000,  1.0000,  4.2922,  ..., -0.5007,  0.2014, -0.3354]) Input Length:  4399
Output:  tensor([0.2500])


### Network ###

In [32]:
import torch.nn as nn
import torch.nn.functional as F

class Network(nn.Module):

    def __init__(self):
        super().__init__()
        
        self.fc1 = nn.Linear(len(train_dataset[0]['inp']), 1600)
        self.b1 = nn.BatchNorm1d(1600)
        self.fc2 = nn.Linear(1600, 1)
        self.softmax = nn.Softmax(dim = 0) 


    def forward(self,x):

        x = self.fc1(x)
        #x = self.b1(x)
        x = self.fc2(x)

        return x
        #return self.softmax(x)
        
    
net = Network()
print(net)
#net.to(torch.device("cuda:0"))
train_dataset[0]

Network(
  (fc1): Linear(in_features=4399, out_features=1600, bias=True)
  (b1): BatchNorm1d(1600, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=1600, out_features=1, bias=True)
  (softmax): Softmax(dim=0)
)


{'inp': tensor([ 5.0000,  2.0000,  2.6844,  ...,  0.0829, -0.5269,  0.1076]),
 'out': tensor([0.2143])}

In [33]:
def train(model, x, y, optimizer, criterion):
    model.zero_grad()
    output = model(x)
    loss = criterion(output,y)
    loss.backward()
    optimizer.step()

    return loss, output


In [34]:
print(torch.cuda.is_available())
print(torch.cuda.current_device())

True
0


### Mean Squared Error ###
Training phase

In [36]:
from torch.optim import Adam
from torch.utils.data import DataLoader
from tqdm import tqdm
import time

criterion = nn.MSELoss()
EPOCHS = 30
BATCH_SIZE = 64
optm = Adam(net.parameters(), lr = 0.00001)

data_train = DataLoader(dataset = train_dataset, batch_size = BATCH_SIZE, shuffle = True)

for epoch in range(EPOCHS):
    epoch_loss = 0
    correct = 0
    
    for bidx, batch in enumerate(data_train):
        #start = time.time()
        x_train = batch['inp']
        y_train = batch['out']
        #print("Fetch time: {}".format(time.time() - start))
        
        #start = time.time()
        loss, predictions = train(net,x_train,y_train, optm, criterion)
        epoch_loss += loss
        #print("Predict time: {}".format(time.time() - start))
        
    print('Epoch {} Loss : {}'.format((epoch+1),epoch_loss))

Epoch 1 Loss : 1.1088427305221558
Epoch 2 Loss : 0.9720829129219055
Epoch 3 Loss : 0.939026951789856
Epoch 4 Loss : 0.9022021889686584
Epoch 5 Loss : 0.9036809802055359
Epoch 6 Loss : 0.859229564666748
Epoch 7 Loss : 0.8599917888641357
Epoch 8 Loss : 0.8424505591392517
Epoch 9 Loss : 0.8321893215179443
Epoch 10 Loss : 0.8420288562774658
Epoch 11 Loss : 0.828995406627655
Epoch 12 Loss : 0.817345380783081
Epoch 13 Loss : 0.811475396156311
Epoch 14 Loss : 0.8073127865791321
Epoch 15 Loss : 0.7900136709213257
Epoch 16 Loss : 0.8010743856430054
Epoch 17 Loss : 0.7903727293014526
Epoch 18 Loss : 0.7630934715270996
Epoch 19 Loss : 0.7665468454360962
Epoch 20 Loss : 0.7612242698669434
Epoch 21 Loss : 0.7631808519363403
Epoch 22 Loss : 0.7604466080665588
Epoch 23 Loss : 0.7576807141304016
Epoch 24 Loss : 0.7574282884597778
Epoch 25 Loss : 0.7654207348823547
Epoch 26 Loss : 0.7408734560012817
Epoch 27 Loss : 0.725262463092804
Epoch 28 Loss : 0.7236118912696838
Epoch 29 Loss : 0.7249473929405212


### Output for single sample

In [37]:
net(train_dataset[210]['inp'])

tensor([0.2395], grad_fn=<AddBackward0>)

### Mean Absolute Error ###

#### MAE for test dataset

In [38]:
from sklearn.metrics import mean_absolute_error

y_true = [test_dataset[i]['out'].item() for i in range(len(test_dataset))]
y_pred = []

test_loader = DataLoader(dataset = test_dataset, batch_size = BATCH_SIZE, shuffle = False)
for bidx, batch in enumerate(test_loader):
        #start = time.time()
        x_train = batch['inp']
        y_pred.append(net(x_train))

y_pred = [x.item() for i in range(len(y_pred)) for x in y_pred[i] ]

mae = mean_absolute_error(y_true, y_pred)
print("MAE for test data: ", mae)

MAE for test data:  0.07294992398726576


#### MAE for trial dataset

In [78]:
from sklearn.metrics import mean_absolute_error

y_true = [trial_dataset[i]['out'].item() for i in range(len(trial_dataset))]
y_pred = []

trial_loader = DataLoader(dataset = trial_dataset, batch_size = BATCH_SIZE, shuffle = False)
for bidx, batch in enumerate(trial_loader):
        #start = time.time()
        x_train = batch['inp']
        y_pred.append(net(x_train))

y_pred = [x.item() for i in range(len(y_pred)) for x in y_pred[i] ]

mae = mean_absolute_error(y_true, y_pred)
print("MAE for trial data: ", mae)

MAE for trial data:  0.07172998721377301


#### MAE for train dataset

In [40]:
from sklearn.metrics import mean_absolute_error

y_true = [train_dataset[i]['out'].item() for i in range(len(train_dataset))]
y_pred = []

test_loader = DataLoader(dataset = train_dataset, batch_size = BATCH_SIZE, shuffle = False)
for bidx, batch in enumerate(test_loader):
        #start = time.time()
        x_train = batch['inp']
        y_pred.append(net(x_train))

y_pred = [x.item() for i in range(len(y_pred)) for x in y_pred[i] ]

mae = mean_absolute_error(y_true, y_pred)
print("MAE for train data: ", mae)

MAE for train data:  0.05870204062072968


#### MAE for total random

In [41]:
from sklearn.metrics import mean_absolute_error
import random

y_true = [train_dataset[i]['out'].item() for i in range(len(train_dataset))]
y_pred = [random.random() for i in range(len(train_dataset))]


mae = mean_absolute_error(y_true, y_pred)
print("Mean Absolute Error for train data: ", mae)

Mean Absolute Error for train data:  0.3059637870602539


#### Demo

In [126]:
def prepare_sentence(sentence, token):
    sentence_embeddings = get_embedding_for_context(sentence)[0]
    handcrafted_features = get_handcrafted_features(token)
            
    if token.lower() in glove_w2v:   
        w2v_for_token = glove_w2v[token.lower()]
    else:
       w2v_for_token = [0] * 300
    
    return {
            'inp': torch.from_numpy(np.hstack((np.array(handcrafted_features), sentence_embeddings, np.array(w2v_for_token))).ravel()).float() 
           }

    

In [127]:
sentence = 'Peter loves pineapples and apples!'
token1 = 'pineapples'
token2 = 'apples'
token3 = 'Peter'

y_pred1 = net(prepare_sentence(sentence, token1)['inp'])
print('SENTENCE: ', sentence, '\nTOKEN: ', token1, '\nCOMPLEXITY: ', y_pred1.item(), '\n')

y_pred2 = net(prepare_sentence(sentence, token2)['inp'])
print('SENTENCE: ', sentence, '\nTOKEN: ', token2, '\nCOMPLEXITY: ', y_pred2.item(), '\n')



SENTENCE:  Peter loves pineapples and apples! 
TOKEN:  pineapples 
COMPLEXITY:  0.33662906289100647 

SENTENCE:  Peter loves pineapples and apples! 
TOKEN:  apples 
COMPLEXITY:  0.1939375251531601 

